This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/lyapunov.html).  I recommend having both windows open, side-by-side!


In [1]:
import matplotlib.pyplot as plt
import mpld3
import numpy as np
from IPython.display import display
from pydrake.all import (
    Jacobian,
    MathematicalProgram,
    Solve,
    Variable,
    Variables,
)
from pydrake.symbolic import Polynomial

from underactuated import running_as_notebook

if running_as_notebook:
    mpld3.enable_notebook()

# Outer approximation for the cubic polynomial

In [5]:
def cubic_outer():
    prog = MathematicalProgram()
    x = prog.NewIndeterminates(1, "x")[0]
    f = -x + x**3

    # Make B(x) a free polynomial.
    B = prog.NewEvenDegreeFreePolynomial(Variables([x]), 4, "b").ToExpression()

    # Bdot(x) negative definite
    Bdot = B.Jacobian([x]).dot(f)[0]
    prog.AddSosConstraint(-Bdot + 0.1 * x**2)

    # B(0) >= 0.1
    prog.AddLinearConstraint(B.Substitute({x: 0}) >= 0.1)

    # Construct W(x) as a SOS polynomial
    # Note: doing it manually to make the integration easier
    W = prog.NewEvenDegreeSosPolynomial(Variables([x]), 4)[0]
    We = W.ToExpression()

    # W(x) > B(x) + 1.0
    prog.AddSosConstraint(We - B - 1.0)

    # min \int_{-2}^2 W(x)dx
    prog.AddCost(W.Integrate(x, -2, 2).ToExpression())

    result = Solve(prog)

    assert result.is_success()

    Bsol = result.GetSolution(B)
    print(f"B = {Bsol}")
    Wsol = result.GetSolution(W)
    fig, ax = plt.subplots()
    xs = np.linspace(-2, 2, 51)
    fsols = [f.Evaluate({x: xi}) for xi in xs]
    Bsols = [Bsol.Evaluate({x: xi}) for xi in xs]
    Wsols = [Wsol.Evaluate({x: xi}) for xi in xs]
    ax.plot(xs, 0 * xs, "k")
    ax.plot(xs, fsols)[0].set_label("f(x)")
    ax.plot(xs, Bsols)[0].set_label("B(x)")
    ax.plot(xs, Wsols)[0].set_label("W(x)")
    plt.ylim(-2.5, 2.5)
    ax.legend()
    display(mpld3.display())


cubic_outer()

B = (0.10000000046721777 + 0.03529968767109825 * pow(x(0), 2) - 0.13295669101456054 * pow(x(0), 4))


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e5ec0aeb-d006-4689-a009-180923e76318' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>